# Goals Option 1

## How much are nearby Hospitals and ASCs paid for the same procedure?
Filters for Physician/Other Supplier
Entity Type = O
Provider Type = Ambulatory Surgical Center
CPT = 43249
Filters for Hospital Outpatient
APC = 5302
Compare results within a CBSA

# Stretch Goals

## Extend to other HCPCS/APCs
- Best place to start is to find an APC that is well-populated in the Hospital file
- Next find a well-populated HCPCS from the APC in the Physician/Other Supplier File

## Compare Doctor Utilization for a procedure
- Filters for Physician/Other Supplier
  - Entity Type = I
  - Place of Service = O
- HCPCS = 99213 (or any office visit – 99211-99215 / 99201-99205)
- How many different patients does each specialty typically see in a year? How does that vary by CBSA?